In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta
import plotly_express as px
import streamlit as st


In [3]:
sum_amount = 1000
start_date = "2023-01-01"
end_date = date.today() + timedelta(days=30)
end_date = end_date.strftime("%Y-%m-%d")

percent_w1, percent_w2, percent_w3 = np.random.dirichlet(np.ones(3))
precent_list = [percent_w1,percent_w2,percent_w3]
 
currency_codes = ["USD", "EUR", "HUF"]
value_list = list(map(lambda x: x * 1000, precent_list))

currency_dict = dict(zip(currency_codes, value_list))




In [4]:

url = f'https://api.nbp.pl/api/exchangerates/tables/a/2023-01-01/2023-01-31/?format=json'
response = requests.get(url)
if response.status_code == 200:
    data_raw = response.json()
else:
    print("Wystąpił błąd:", response.status_code)

In [5]:
list_data = []
list_date = []
for date in data_raw:
    for data in date['rates']:
        list_data.append(list(data.values()))
        list_date.append(date['effectiveDate'])

In [6]:
df_raw = pd.DataFrame(list_data)
df_raw['data'] = list_date
df_raw = df_raw.rename(columns={0:'currency',1:'code',2:'mid'})

In [7]:
df = df_raw.copy()
df = df.query('@currency_codes in code').sort_values(by=['code','data'])

In [8]:
df['converted_value'] = df['code'].map(currency_dict) * df['mid']


In [9]:
df.groupby(by='data').agg({'converted_value': ['sum','mean']})

converted_value             
                       sum         mean
data                                   
2023-01-02     4011.856686  1337.285562
2023-01-03     4030.588368  1343.529456
2023-01-04     4012.427898  1337.475966
2023-01-05     4012.721729  1337.573910
2023-01-09     4028.606169  1342.868723
2023-01-10     4020.012347  1340.004116
2023-01-11     4011.271886  1337.090629
2023-01-12     4009.592138  1336.530713
2023-01-13     4002.099254  1334.033085
2023-01-16     4012.281522  1337.427174
2023-01-17     4007.825633  1335.941878
2023-01-18     4013.551516  1337.850505
2023-01-19     4016.087867  1338.695956
2023-01-20     4022.410906  1340.803635
2023-01-23     4016.334552  1338.778184
2023-01-24     4013.139060  1337.713020
2023-01-25     4011.090051  1337.030017
2023-01-26     4022.882700  1340.960900
2023-01-27     4010.513513  1336.837838
2023-01-30     4015.395718  1338.465239
2023-01-31     4018.477788  1339.492596